In [ ]:
from rtlsdr import RtlSdr
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from scipy.signal import correlate

from plotly import graph_objects as go

from util.plotting import compute_fft_plot, compute_fft_plot_from_sample_rate, compute_frequency_response
from util.filtering import low_pass_filter_complex_signal, band_pass_filter_real_signal, low_pass_filter_real_signal
from util.data_io import read_gqrx_raw_data
from util.demodulation import demodulate_signal, chunked_demodulate_signal


# from scipy.signal import hilbert, resample, get_window, spectrogram
# from scipy.fftpack import fftshift
# import plotly.graph_objects as go

In [ ]:
fs = 1.8E6
center_frequency = 144.390E6

signal = read_gqrx_raw_data("data/gqrx_20211128_230741_144390000_1800000_fc.raw")
len(signal) / fs

In [ ]:
filtered_signal = low_pass_filter_complex_signal(signal=signal, cutoff_frequency=10E3, sample_rate=fs)

angle_diff = np.angle(
    np.conjugate(filtered_signal[:-1]) * filtered_signal[1:]
)

filtered_signal = band_pass_filter_real_signal(
    signal=angle_diff, low_cutoff_frequency=1000, high_cutoff_frequency=2500,
    sample_rate=fs
)
filtered_signal = filtered_signal[::30]
downsampled_fs = fs / 30



In [ ]:
downsampled_fs

In [ ]:
f,m = compute_fft_plot_from_sample_rate(filtered_signal, downsampled_fs)

# plt.plot(f,m)
fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
space_signal_filtered = band_pass_filter_real_signal(
    signal=filtered_signal, low_cutoff_frequency=1150, high_cutoff_frequency=1250,
    sample_rate=downsampled_fs, order=50
)
space_envelope = low_pass_filter_real_signal(np.abs(space_signal_filtered), cutoff_frequency=1.2E3,
                                             sample_rate=downsampled_fs, apply_mean_shift=False)

mark_signal_filtered = band_pass_filter_real_signal(
    signal=filtered_signal, low_cutoff_frequency=2150, high_cutoff_frequency=2250,
    sample_rate=downsampled_fs, order=50
)
mark_envelope = low_pass_filter_real_signal(np.abs(mark_signal_filtered), cutoff_frequency=1.2E3,
                                            sample_rate=downsampled_fs, apply_mean_shift=False)

decision_signal = mark_envelope - space_envelope

In [ ]:
fig = go.Figure()

f,m = compute_fft_plot_from_sample_rate(space_signal_filtered, downsampled_fs)
fig.add_scatter(x=f,y=m, name="space")

f,m = compute_fft_plot_from_sample_rate(mark_signal_filtered, downsampled_fs)
fig.add_scatter(x=f,y=m, name="mark")

fig.show()

In [ ]:
fig = go.Figure()

time = np.arange(len(space_signal_filtered)) / downsampled_fs
fig.add_scatter(x=time,y=space_signal_filtered, name="space")
fig.add_scatter(x=time,y=space_envelope, name="space envelope")

fig.add_scatter(x=time,y=mark_signal_filtered, name="mark")
fig.add_scatter(x=time,y=mark_envelope, name="mark envelope")

fig.add_scatter(x=time,y=decision_signal, name="decision")


fig.show()

In [ ]:
mark_frequency = 2225
space_frequency = 1176

mark_time = np.arange(downsampled_fs/mark_frequency+1) / downsampled_fs
mark_cos = np.cos(2*np.pi*mark_frequency*mark_time)
mark_sin = np.sin(2*np.pi*mark_frequency*mark_time)

space_time = np.arange(downsampled_fs/space_frequency+1) / downsampled_fs
space_cos = np.cos(2*np.pi*space_frequency*space_time)
space_sin = np.sin(2*np.pi*space_frequency*space_time)

In [ ]:
fig = go.Figure()
fig.add_scatter(x=mark_time,y=mark_cos, name="mark cos")
fig.add_scatter(x=mark_time,y=mark_sin, name="mark sin")

fig.add_scatter(x=space_time,y=space_sin, name="space sin")
fig.add_scatter(x=space_time,y=space_cos, name="space cos")

fig.show()

In [ ]:
mark_cos_correlation = correlate(filtered_signal, mark_cos, mode="same")
mark_sin_correlation = correlate(filtered_signal, mark_sin, mode="same")

mark_correlation = np.sqrt(mark_cos_correlation**2 + mark_sin_correlation**2)
mark_correlation /= np.max(mark_correlation)

space_cos_correlation = correlate(filtered_signal, space_cos, mode="same")
space_sin_correlation = correlate(filtered_signal, space_sin, mode="same")

space_correlation = np.sqrt(space_cos_correlation**2 + space_sin_correlation**2)
space_correlation /= np.max(space_correlation)

decision_signal = mark_correlation - space_correlation
decision_signal -= np.mean(decision_signal)

In [ ]:
len(mark_correlation), len(space_correlation)

In [ ]:
time = np.arange(len(mark_correlation)) / downsampled_fs

fig = go.Figure()
fig.add_scatter(x=time,y=mark_correlation, name="mark")
fig.add_scatter(x=time,y=space_correlation, name="space")
fig.add_scatter(x=time,y=decision_signal, name="decision")
fig.show()

In [ ]:
f,m = compute_fft_plot_from_sample_rate(decision_signal, downsampled_fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()


In [ ]:
filtered_signal = low_pass_filter_complex_signal(signal=signal, cutoff_frequency=10E3, sample_rate=fs)
filtered_signal = filtered_signal[::10]
downsampled_fs = fs / 10


angle_diff = np.angle(
    np.conjugate(filtered_signal[:-1]) * filtered_signal[1:]
)

filtered_angle_diff = band_pass_filter_real_signal(
    signal=angle_diff, low_cutoff_frequency=1000, high_cutoff_frequency=2500,
    sample_rate=downsampled_fs
)
filtered_angle_diff = filtered_angle_diff[::10]
downsampled_fs = downsampled_fs / 10


In [ ]:
len(filtered_angle_diff) / downsampled_fs
int(downsampled_fs * 0.5), int(downsampled_fs * 1.5)

In [ ]:
wavfile.write("data/aprs.wav", int(downsampled_fs), filtered_angle_diff)

In [ ]:
f,m = compute_fft_plot_from_sample_rate(filtered_angle_diff, downsampled_fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()
# plt.plot(f,m)

In [ ]:
# shift the frequencies to be centered at zero.
time_vector = np.linspace(0, len(filtered_angle_diff) / downsampled_fs, len(filtered_angle_diff))
shift_frequency = 1700
shift_signal = np.exp(-2j*np.pi*shift_frequency*time_vector)

centered_signal = filtered_angle_diff * shift_signal

centered_signal = low_pass_filter_complex_signal(
    signal=centered_signal, cutoff_frequency=1000, sample_rate=downsampled_fs
)


In [ ]:
f,m = compute_fft_plot_from_sample_rate(centered_signal, downsampled_fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
angle_diff = np.angle(
    np.conjugate(centered_signal[:-1]) * centered_signal[1:]
)

angle_diff = low_pass_filter_complex_signal(
    signal=angle_diff, cutoff_frequency=1500, sample_rate=downsampled_fs
)

In [ ]:
f,m = compute_fft_plot_from_sample_rate(angle_diff, downsampled_fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:

fig = go.Figure()
fig.add_scatter(x=time_vector,y=np.real(angle_diff))
fig.show()

In [ ]:
zero_frequency = 1200
one_frequency = 2200

times = np.linspace(0, len(demodulated_audio) / audio_fs, len(demodulated_audio))

zero_signal_cos = np.cos(2*np.pi*times*zero_frequency)
zero_signal_sin = np.sin(2*np.pi*times*zero_frequency)

one_signal_cos = np.cos(2*np.pi*times*one_frequency)
one_signal_sin = np.sin(2*np.pi*times*one_frequency)

In [ ]:
f,m = compute_fft_plot_from_sample_rate(zero_signal_cos + one_signal_cos, audio_fs)
plt.plot(f,m)

In [ ]:
fs, signal = wavfile.read("data/SDRSharp_20150819_190253Z_145070kHz_IQ.wav")

signal = (signal[:, 0] + 1j * signal[:, 1]) / np.mean(np.abs(signal))

In [ ]:
f,m = compute_fft_plot_from_sample_rate(signal, fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
filtered_signal = low_pass_filter_complex_signal(signal=signal, cutoff_frequency=10E3, sample_rate=fs)


angle_diff = np.angle(
    np.conjugate(filtered_signal[:-1]) * filtered_signal[1:]
)

filtered_angle_diff = band_pass_filter_real_signal(
    signal=angle_diff, low_cutoff_frequency=1000, high_cutoff_frequency=2500,
    sample_rate=fs
)


In [ ]:
f,m = compute_fft_plot_from_sample_rate(filtered_angle_diff, fs)

fig = go.Figure()
fig.add_scatter(x=f,y=m)
fig.show()

In [ ]:
# shift the frequencies to be centered at zero.
time_vector = np.linspace(0, len(filtered_angle_diff) / fs, len(filtered_angle_diff))
shift_frequency = 1700
shift_signal = np.exp(-2j*np.pi*shift_frequency*time_vector)

centered_signal = filtered_angle_diff * shift_signal

centered_signal = low_pass_filter_complex_signal(
    signal=centered_signal, cutoff_frequency=1000, sample_rate=fs
)


angle_diff = np.angle(
    np.conjugate(centered_signal[:-1]) * centered_signal[1:]
)

angle_diff = low_pass_filter_complex_signal(
    signal=angle_diff, cutoff_frequency=1500, sample_rate=fs
)

In [ ]:
st = int(fs * 1)
end = int(fs * 10)


fig = go.Figure()
fig.add_scatter(x=time_vector[st:end],y=np.real(angle_diff[st:end]))
fig.show()